In [1]:
import torch
import sys

import coremltools as ct

sys.path.append("..")

from app.model import SleepModel

model = SleepModel.load_from_checkpoint(
    "../lightning_logs/version_16/checkpoints/epoch=9-step=3990.ckpt"
)
model.eval()


Torch version 1.13.0 has not been tested with coremltools. You may run into unexpected errors. Torch 1.12.1 is the most recent version that has been tested.


SleepModel(
  (net): SleepNet(
    (layers): Sequential(
      (0): Linear(in_features=13, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): GELU(approximate='none')
      (4): Dropout(p=0.1, inplace=False)
      (5): Linear(in_features=128, out_features=128, bias=True)
      (6): GELU(approximate='none')
      (7): Dropout(p=0.1, inplace=False)
      (8): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (net_ema): EMAModel(
    (module): SleepNet(
      (layers): Sequential(
        (0): Linear(in_features=13, out_features=128, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): GELU(approximate='none')
        (4): Dropout(p=0.1, inplace=False)
        (5): Linear(in_features=128, out_features=128, bias=True)
        (6): GELU(approximate='none')
        (7): Dropout(p=0.1, inplace=False)
        (8): Linea

In [2]:
accs, hvs, hds = torch.randn(1, 3), torch.randn(1, 5), torch.randn(1, 5)
traced = torch.jit.trace(model.net_ema.module, (accs, hvs, hds))

In [3]:
mlmodel = ct.convert(
    traced,
    inputs=[
        ct.TensorType(shape=accs.shape, name="accs"),
        ct.TensorType(shape=hvs.shape, name="hvs"),
        ct.TensorType(shape=hds.shape, name="hds"),
    ],
    outputs=[ct.TensorType(name="out")],
    # classifier_config=ct.ClassifierConfig(["awake", "asleep"]),
)


Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 27/27 [00:00<00:00, 1447.57 ops/s]


In [4]:
mlmodel.predict({"accs": accs, "hvs": hvs, "hds": hds})

{'out': array([0.4663086], dtype=float32)}

In [5]:
mlmodel.save("sleep.mlmodel")